In [1]:
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import GetHistoryRequest
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
import re

In [2]:
api_id = '28626741'
api_hash = 'bccf833966cbe42d21d420fd3c3d6457'
phone = '+251936679199'  

client = TelegramClient('session_name', api_id, api_hash)

In [3]:
async def connect_to_telegram():
    """Connects to Telegram and handles authentication."""
    await client.start(phone)
    print("Connected to Telegram!")
    
    # Check if user is authorized
    if not await client.is_user_authorized():
        try:
            await client.send_code_request(phone)
            code = input("Enter the code sent to your Telegram: ")
            await client.sign_in(phone, code)
        except SessionPasswordNeededError:
            password = input("Enter your password: ")
            await client.sign_in(password=password)


In [4]:
# Connect to Telegram
await connect_to_telegram()


Connected to Telegram!


In [5]:
async def fetch_messages(channel_username, limit=100):
    """Fetches messages from a Telegram channel."""
    try:
        channel = await client.get_entity(channel_username)
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=0,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))
        return history.messages
    except Exception as e:
        print(f"Error fetching messages: {e}")
        return []

In [6]:
async def list_channels():
    """Lists all channels the user is a part of."""
    dialogs = await client.get_dialogs()
    channels = [dialog for dialog in dialogs if dialog.is_channel]
    for channel in channels:
        print(f"Channel Name: {channel.name}, Username: @{channel.entity.username}")

await list_channels()


Channel Name: Zabbix, Username: @None
Channel Name: AudioBook Collection • Audiobooks Archive • Ebooks, Username: @Audiobooks_collection
Channel Name: Bridal Team only, Username: @None
Channel Name: Fullstack web developers, Username: @reactjs2020
Channel Name: RUHAMAH🔥, Username: @ruhamahfam
Channel Name: USED PHONE SELLER™ 🇪🇹, Username: @usedphoneselleres
Channel Name: Child Dev't Network, Username: @childdevtnetwork
Channel Name: TIKVAH-ETHIOPIA, Username: @tikvahethiopia
Channel Name: TV Series Bot 📺, Username: @Netflix_Xseriesbot
Channel Name: DINOPUTIH | #VIRAL, Username: @LYDIA_ONIC_VIRAL_xu
Channel Name: Khalid chiffon, Username: @None
Channel Name: TIKVAH-MAGAZINE, Username: @tikvahethmagazine
Channel Name: Google Facts™ [ ️@googlefactss🌎], Username: @googlefactss
Channel Name: TIKVAH-CART, Username: @tikvahethcart
Channel Name: Afriwork (Freelance Ethiopia), Username: @freelance_ethio
Channel Name: شرکت افزایش سرمایه گذاری خوارزمی ♻️توسعه صنعتی♻️ 🔷️[ سهامی عام ]🔷️, Username: 

In [7]:
channels = ['@googlefactss', '@Audiobooks_collection', '@reactjs2020']

In [8]:
all_messages = []
for channel in channels:
    messages = await fetch_messages(channel, limit=500)
    all_messages.extend(messages)


In [9]:
# Process and Clean Messages
def process_messages(messages):
    """Processes and cleans messages."""
    data = []
    for msg in messages:
        if msg.message:  # Check if the message contains text
            data.append({
                'sender': msg.from_id.user_id if msg.from_id else None,
                'date': msg.date,
                'text': msg.message
            })
    return pd.DataFrame(data)

In [10]:
# Convert Messages to DataFrame
df = process_messages(all_messages)

In [11]:
def clean_text(text):
    """Cleans text data: removes special characters, links, and tokenizes."""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^ሀ-፡a-zA-Z0-9]", " ", text)  # Keep Amharic and English characters
    text = text.lower()
    return text

In [12]:
df['cleaned_text'] = df['text'].apply(clean_text)

In [13]:
df.to_csv('telegram_messages.csv', index=False)
print("Preprocessed data saved to 'telegram_messages.csv'.")

Preprocessed data saved to 'telegram_messages.csv'.


In [14]:
# Exploratory Data Analysis (EDA)
# Word Frequency Analysis
from collections import Counter

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tselotbeyene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
from telethon.errors.rpcerrorlist import ServerError

async def safe_connect():
    retries = 3
    while retries > 0:
        try:
            await client.connect()
            if await client.is_user_authorized():
                print("Connected to Telegram!")
                break
        except ServerError as e:
            print(f"Connection failed: {e}. Retrying...")
            retries -= 1
            await asyncio.sleep(5)
    if retries == 0:
        print("Failed to connect after multiple attempts.")




In [17]:
print(df.head())

   sender                      date  \
0     NaN 2025-01-20 19:17:32+00:00   
1     NaN 2025-01-20 14:02:29+00:00   
2     NaN 2025-01-20 10:59:56+00:00   
3     NaN 2025-01-20 07:35:12+00:00   
4     NaN 2025-01-19 17:07:13+00:00   

                                                text  \
0  Australia is the only continent without any gl...   
1  If a Catholic priest reveals anything someone ...   
2  The word breakfast comes from the Middle Engli...   
3  The word “honeymoon” originated from the tradi...   
4  There is a positive correlation between vitami...   

                                        cleaned_text  
0  australia is the only continent without any gl...  
1  if a catholic priest reveals anything someone ...  
2  the word breakfast comes from the middle engli...  
3  the word  honeymoon  originated from the tradi...  
4  there is a positive correlation between vitami...  


In [30]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Download if needed: python -m spacy download en_core_web_sm
doc = nlp("This is a test sentence for tokenization.")
tokens = [token.text for token in doc]
print(tokens)


['This', 'is', 'a', 'test', 'sentence', 'for', 'tokenization', '.']
